In [1]:
pip install numpy pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import glob # finding file paths by pattern (e.g. "*.csv")
import os # working with folder/file paths
import re # For regular expressions (to pull numbers out of filenames)
from collections import OrderedDict # OrderedDict remembers the order that you insert items into
from IPython.display import display # display() nicely shows Dataframes

In [3]:
release_points = {
    'test-2': {'X': -4.856481129, 'Z': -4.322118356},
    'test-3': {'X': -4.89719112, 'Z': -4.183614366},
    'test-4': {'X': -5.186691303, 'Z': -4.322118356}, 
    'test-5': {'X': -1.762650653, 'Z': -0.57249375}, 
    'test-6': {'X': -8.744156604, 'Z': -1.08049375},
    'test-7': {'X': -7.538065163, 'Z': -5.16671875}, 
    'test-8': {'X': -9.175183123, 'Z': -7.98929375},
    'test-9': {'X': -3.326592432, 'Z': -8.01469375}, 
    'test-10': {'X': -0.974541231, 'Z': -4.02689375},
    'test-11': {'X': -5.186691303, 'Z': -4.322118356},
    'test-12': {'X':-4.856481129, 'Z': -4.322118356},
}

# LOAD RAW DATA

In [4]:
custom_bounds = {
    'test-11': {'col': 'flow-time', 'low': 60, 'high': 1060},
    'test-12': {'col': 'flow-time', 'low': 0, 'high': 1000},
}

In [5]:
# Apply these to any files with "time" or "flow-time" columns
defaults = {
    'time': (0, 1000),
    'flow-time': (300, 1300),
}

In [6]:
# Function to pull the first number from a filename
def extract_num(path):
    # get "test-3.csv" -> "test-3"
    name = os.path.splitext(os.path.basename(path))[0]
    # look for digits inside that base name
    m = re.search(r'(\d+)', name)
    # Return number if found, else a big "infinite" value
    return int(m.group(1)) if m else float('inf')



In [7]:
# Find all CSVs and sort by that first number
data_folder = os.path.join('..', '..', 'Datasets')
pattern = os.path.join(data_folder, '*.csv')
paths = sorted(glob.glob(pattern), key=extract_num) # paths is a list of full CSV file paths, in numeric order

In [8]:
# Load each CSV, filter, tag with X/Z, and keep in order
filtered = OrderedDict()
for path in paths:
    name = os.path.splitext(os.path.basename(path))[0]
    df = pd.read_csv(path)

    # Choose which column & bounds to use
    if name in custom_bounds:
        spec = custom_bounds[name]
        col, low, high = spec['col'], spec['low'], spec['high']
    elif 'time' in df.columns:
        col, (low, high) = 'time', defaults['time']

    elif 'flow-time' in df.columns:
        col, (low, high) = 'flow-time', defaults['flow-time']

    else:

        continue

    # actually filter those rows and reset row numbers
    df_range = df[df[col].between(low, high)].copy()
    df_range.reset_index(drop=True, inplace=True)

    if name in release_points:
        df_range['X'] = release_points[name]['X']
        df_range['Z'] = release_points[name]['Z']

    else:
        print(f"no release_points entry for {name}")

    # Store under a new key like "test2_data_range"
    filtered[f"{name}_range"] = df_range


def head_tail(df, n = 5):
    # take first n rows and last n rows, put them together
    return pd.concat([df.head(n), df.tail(n)])

# Show just the first and last 5 rows
for key, df_r in filtered.items():
    print(f"{key}: {len(df_r)} rows -> ")
    print('---------------------------')

    display(head_tail(df_r, n=5))
    #display(df_r.tail())

test-2_range: 1001 rows -> 
---------------------------


,time,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,X,Z
0,0,-107.526882,-226.000000,222.222222,175.627240,268.817204,-14.336918,86.021505,-57.347670,-78.853047,107.526882,-688.172043,-4505.376344,-4856.630824,258.064516,-408.602150,-25.089606,-4.856481,-4.322118
1,1,-107.526882,-151.000000,71.684588,175.627240,193.548387,60.931900,10.752688,-57.347670,-78.853047,32.258065,-612.903226,-4430.107527,-4781.362007,258.064516,-333.333333,-100.358423,-4.856481,-4.322118
2,2,-107.526882,-75.300000,71.684588,250.896057,193.548387,60.931900,10.752688,-57.347670,-78.853047,107.526882,-537.634409,-4354.838710,-4630.824373,182.795699,-333.333333,-25.089606,-4.856481,-4.322118
3,3,-107.526882,-150.537634,222.222222,326.164875,268.817204,-14.336918,161.290323,-57.347670,-78.853047,107.526882,-537.634409,-4354.838710,-4706.093190,182.795699,-258.064516,-25.089606,-4.856481,-4.322118
4,4,-32.258065,-75.268817,71.684588,326.164875,193.548387,60.931900,311.827957,-57.347670,-154.121864,107.526882,-537.634409,-4279.569892,-4630.824373,107.526882,-182.795699,50.179211,-4.856481,-4.322118
996,996,2526.881720,4290.322581,4362.007168,5218.637993,3731.182796,3146.953405,2494.623656,2878.136201,2103.942652,3946.236559,440.860215,2870.967742,2896.057348,2967.741935,4483.870968,4114.695341,-4.856481,-4.322118
997,997,2451.612903,4440.860215,4286.738351,5218.637993,3806.451613,3146.953405,2569.892473,2802.867384,2103.942652,3870.967742,440.860215,2569.892473,2519.713262,2892.473118,4408.602151,4265.232975,-4.856481,-4.322118
998,998,2451.612903,4365.591398,4286.738351,5293.906810,3806.451613,3222.222222,2645.161290,2878.136201,2179.211470,3795.698925,440.860215,2419.354839,1992.831541,2892.473118,4182.795699,4491.039427,-4.856481,-4.322118
999,999,2451.612903,4365.591398,4286.738351,5369.175627,3806.451613,3222.222222,2645.161290,2878.136201,2179.211470,3870.967742,741.935484,1967.741935,1616.487455,2817.204301,4107.526882,4867.383513,-4.856481,-4.322118
1000,1000,2451.612903,4365.591398,4211.469534,5369.175627,3806.451613,3372.759857,2569.892473,2878.136201,2254.480287,3870.967742,1043.010753,1591.397849,1390.681004,2741.935484,3881.720430,4942.652330,-4.856481,-4.322118


test-3_range: 1001 rows -> 
---------------------------


,time,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,X,Z
0,0,30,-170,30,-70,-40,20,-100,120,50,60,70,50,100,40,70,120,-4.897191,-4.183614
1,1,0,60,60,-130,-140,20,0,60,50,-20,50,60,60,-220,30,0,-4.897191,-4.183614
2,2,40,-150,60,-100,-190,40,50,20,70,100,10,40,70,-160,100,170,-4.897191,-4.183614
3,3,30,-150,40,60,-120,60,20,40,80,100,40,80,50,-170,70,10,-4.897191,-4.183614
4,4,60,-240,40,-110,-440,50,-20,-100,70,40,110,10,70,-130,30,40,-4.897191,-4.183614
996,996,490,380,590,740,2690,1370,1210,920,450,420,460,530,3060,2070,1680,1600,-4.897191,-4.183614
997,997,450,190,600,600,1710,2200,1110,900,430,440,530,490,3060,2100,1620,1670,-4.897191,-4.183614
998,998,480,210,590,530,3910,2440,1170,860,440,480,480,480,3010,1610,1620,1580,-4.897191,-4.183614
999,999,570,330,630,670,3330,2400,1180,860,440,470,450,510,3070,1940,1650,1570,-4.897191,-4.183614
1000,1000,670,290,690,910,3400,1510,1270,950,470,430,460,560,3120,1990,1740,1750,-4.897191,-4.183614


test-4_range: 1001 rows -> 
---------------------------


...

,flow-time,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,X,Z
0,300.0000,17.752182,18.488436,17.717110,18.606426,18.790766,19.144455,18.663036,19.225666,15.453478,16.391560,16.297522,15.987082,16.280884,17.253485,15.222475,14.489625,-0.974541,-4.026894
1,300.0001,17.752193,18.488432,17.717108,18.606421,18.790768,19.144451,18.663027,19.225668,15.453477,16.391560,16.297520,15.987082,16.280882,17.253473,15.222475,14.489623,-0.974541,-4.026894
2,300.0002,17.752203,18.488428,17.717106,18.606417,18.790768,19.144449,18.663019,19.225670,15.453477,16.391560,16.297520,15.987078,16.280882,17.253466,15.222475,14.489621,-0.974541,-4.026894
3,300.0003,17.752211,18.488428,17.717106,18.606413,18.790768,19.144449,18.663013,19.225672,15.453475,16.391560,16.297520,15.987078,16.280880,17.253452,15.222475,14.489619,-0.974541,-4.026894
4,300.0004,17.752218,18.488424,17.717106,18.606411,18.790768,19.144449,18.663006,19.225674,15.453475,16.391560,16.297520,15.987077,16.280878,17.253445,15.222475,14.489616,-0.974541,-4.026894
112796,1299.9501,708.754883,633.302368,717.608032,965.391846,616.543762,486.033081,525.522522,596.936829,672.941162,761.788208,855.351074,699.518860,687.185913,781.484680,479.829254,468.956848,-0.974541,-4.026894
112797,1299.9601,708.707642,633.333313,717.549927,965.349731,616.581238,486.033173,525.527832,596.930786,672.922668,761.699402,855.358215,699.542053,687.204468,781.491760,479.861847,468.984619,-0.974541,-4.026894
112798,1299.9701,708.660400,633.364624,717.491943,965.307800,616.618958,486.033325,525.533264,596.924622,672.904419,761.610718,855.365356,699.565430,687.223145,781.499329,479.894409,469.012390,-0.974541,-4.026894
112799,1299.9801,708.613037,633.396301,717.433777,965.265686,616.656982,486.033478,525.538574,596.918335,672.886047,761.522095,855.372681,699.588806,687.241821,781.507263,479.927155,469.040192,-0.974541,-4.026894
112800,1299.9901,708.565613,633.428650,717.375793,965.223694,616.695129,486.033630,525.544006,596.911987,672.867676,761.433655,855.379822,699.612366,687.260498,781.515747,479.959808,469.068115,-0.974541,-4.026894


test-11_range: 158357 rows -> 
---------------------------


,flow-time,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,X,Z
0,60.0044,0,0.000334,0.000320,0.000413,0.000460,0.000082,0.000400,0.000443,0.000089,0.222600,0.219254,0.000063,0.000466,0.000279,0.000168,0.000402,-5.186691,-4.322118
1,60.0144,0,0.000334,0.000320,0.000413,0.000460,0.000082,0.000400,0.000443,0.000089,0.222599,0.219571,0.000063,0.000448,0.000280,0.000165,0.000402,-5.186691,-4.322118
2,60.0244,0,0.000334,0.000320,0.000413,0.000460,0.000082,0.000400,0.000443,0.000088,0.222599,0.219886,0.000063,0.000440,0.000275,0.000164,0.000402,-5.186691,-4.322118
3,60.0344,0,0.000334,0.000320,0.000413,0.000460,0.000082,0.000400,0.000443,0.000088,0.222600,0.220200,0.000064,0.000453,0.000275,0.000162,0.000402,-5.186691,-4.322118
4,60.0444,0,0.000334,0.000320,0.000413,0.000460,0.000082,0.000400,0.000443,0.000087,0.222602,0.220514,0.000064,0.000430,0.000277,0.000161,0.000402,-5.186691,-4.322118
158352,1059.9751,0,791.486938,857.347656,804.698181,782.696045,777.723694,968.346802,878.197754,716.407104,641.137146,565.122559,387.138977,353.733185,571.289795,563.687805,631.462952,-5.186691,-4.322118
158353,1059.9801,0,791.434204,857.295593,804.650879,782.664856,777.730225,968.296204,878.228638,716.427673,641.257263,565.125793,387.131805,353.731049,571.305237,563.693787,631.473022,-5.186691,-4.322118
158354,1059.9851,0,791.381348,857.243408,804.603638,782.633789,777.737183,968.245544,878.259277,716.448364,641.377380,565.129028,387.124634,353.728912,571.320618,563.699829,631.483093,-5.186691,-4.322118
158355,1059.9901,0,791.328613,857.191345,804.556152,782.602722,777.743835,968.194885,878.290222,716.468933,641.497620,565.132446,387.117432,353.726868,571.336121,563.705750,631.493103,-5.186691,-4.322118
158356,1059.9951,0,791.275757,857.139160,804.508972,782.571594,777.750854,968.144409,878.321106,716.489624,641.617859,565.135742,387.110260,353.724792,571.351562,563.711670,631.503052,-5.186691,-4.322118


test-12_range: 97972 rows -> 
---------------------------


,flow-time,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,X,Z
0,0.388,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.856481,-4.322118
1,0.398,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.856481,-4.322118
2,0.408,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.856481,-4.322118
3,0.418,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.856481,-4.322118
4,0.428,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.856481,-4.322118
97967,999.958,0.672068,0,1121.263428,1115.966431,1162.688965,1044.396729,1187.366455,1170.466309,4.814718,0.036940,810.533569,3249.333984,2454.925781,3635.814209,4138.591309,4521.295410,-4.856481,-4.322118
97968,999.968,0.672087,0,1121.266357,1115.972534,1162.691162,1044.400146,1187.375244,1170.469360,4.814291,0.036951,810.519714,3249.360840,2454.945068,3635.873291,4138.637207,4521.345215,-4.856481,-4.322118
97969,999.978,0.672072,0,1121.269653,1115.978271,1162.693726,1044.403564,1187.384155,1170.472168,4.813863,0.036962,810.505737,3249.387451,2454.964355,3635.932617,4138.683594,4521.395020,-4.856481,-4.322118
97970,999.988,0.672060,0,1121.272827,1115.984375,1162.696411,1044.406982,1187.393311,1170.474854,4.813434,0.036973,810.491699,3249.414062,2454.983398,3635.991699,4138.729980,4521.444336,-4.856481,-4.322118
97971,999.998,0.672065,0,1121.276001,1115.990234,1162.698730,1044.410522,1187.401978,1170.477783,4.813005,0.036985,810.477661,3249.440674,2455.002441,3636.051025,4138.776367,4521.494141,-4.856481,-4.322118


# Save DATASET FILE

In [9]:
# Make folder
out_dir = 'processed'
os.makedirs(out_dir, exist_ok = True)

In [10]:
# Save each filtered dataset in a numbered, ordered way

for idx, (key, df_r) in enumerate(filtered.items(), start=2):
    filename = f"{idx:02d}_{key}.csv"
    out_path = os.path.join(out_dir, filename)
    df_r.to_csv(out_path, index=False)
    print(f"Saved {out_path}")


Saved processed/02_test-2_range.csv
Saved processed/03_test-3_range.csv
Saved processed/04_test-4_range.csv
Saved processed/05_test-5_range.csv
Saved processed/06_test-6_range.csv
Saved processed/07_test-7_range.csv
Saved processed/08_test-8_range.csv
Saved processed/09_test-9_range.csv
Saved processed/10_test-10_range.csv
Saved processed/11_test-11_range.csv
Saved processed/12_test-12_range.csv


In [11]:
combined = pd.concat(filtered.values(), ignore_index=True)
combined_out = os.path.join(out_dir, '00_all_tests_filtered.csv')
combined.to_csv(combined_out, index=False)

print(f"Saved combined dataset to {combined_out}")

Saved combined dataset to processed/00_all_tests_filtered.csv


In [ ]:
# for key, df_r in filtered.items():
#     out_path = os.path.join(out_dir, f"{key}.csv")
#     df_r.to_csv(out_path, index=False)
#     print(f"Saved {out_path}")